## A Pythonic Card Deck

In [1]:
from collections import namedtuple
from typing import Union

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position: Union[int, slice]):
        return self._cards[position]

In [2]:
beer_cards = Card('7', 'diamonds')
beer_cards

Card(rank='7', suit='diamonds')

In [3]:
deck = FrenchDeck()
len(deck)

52

In [4]:
deck[0]

Card(rank='2', suit='spades')

In [5]:
from random import choice

choice(deck)

Card(rank='Q', suit='spades')

Because our `__getitem__` delegates to the [] operator of self._cards, our deck auto‐
matically supports slicing. 

Just by implementing the `__getitem__` special method, our deck is also iterable:

In [6]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [7]:
i = 0
for card in deck:
    print(card)
    i += 1
    if i == 2:
        break

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')


In [8]:
i = 0
for card in reversed(deck):
    print(card)
    i += 1
    if i == 2:
        break

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')


In [9]:
Card('Q', 'hearts') in deck

True

In [10]:
Card('7', 'beasts') in deck

False

sorting...

In [11]:
suite_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suite_values) + suite_values[card.suit]

In [12]:
i = 0 
for card in sorted(deck, key=spades_high):
    print(card)
    i += 1 
    if i == 3:
        break

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')


## How Special Methods Are Used
The first thing to know about special methods is that they are meant to be called by the
Python interpreter, and not by you. You don’t write my_object.`__len__`(). You write
len(my_object) and, if my_object is an instance of a user-defined class, then Python
calls the `__len__` instance method you implemented

### Emulating Numeric Types
Several special methods allow user objects to respond to operators such as +. We will
cover that in more detail in Chapter 13, but here our goal is to further illustrate the use
of special methods through another simple example.
We will implement a class to represent two-dimensional vectors—that is Euclidean
vectors like those used in math and physics (see Figure 1-1

In [13]:
# Example 1-2. A simple two-dimensional vector class
from math import hypot

class Vector:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self) -> str:
        return 'Vector (%r, %r)' % (self.x, self.y)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [14]:
v1 = Vector(2,4)
v2 = Vector(2,1)
v1 + v2

Vector (4, 5)

In [15]:
v = Vector(3, 4)
abs(v)

5.0

In [16]:
v * 3

Vector (9, 12)

In [17]:
abs(v * 3)

15.0

### String Representation
The `__repr__` special method is called by the repr built-in to get the string represen‐
tation of the object for inspection. If we did not implement `__repr__`, vector instances
would be shown in the console like <Vector object at 0x10e100070>.

The string returned by `__repr__` should be unambiguous and, if possible, match the
source code necessary to re-create the object being represented. That is why our chosen
representation looks like calling the constructor of the class (e.g., Vector(3, 4)).

Contrast `__repr__` with `__str__`, which is called by the str() constructor and implicitly
used by the print function. `__str__` should return a string suitable for display to end
users

If you only implement one of these special methods, choose `__repr__`, because when
no custom `__str__` is available, Python will call `__repr__` as a fallback.

### Arithmetic Operators
Example 1-2 implements two operators: + and *, to show basic usage of `__add__` and
`__mul__`. Note that in both cases, the methods create and return a new instance of
Vector, and do not modify either operand—self or other are merely read. This is the
expected behavior of infix operators: to create new objects and not touch their operands.
I will have a lot more to say about that in Chapter 13

### Boolean Value of a Custom Type

Although Python has a bool type, it accepts any object in a boolean context, such as the
expression controlling an if or while statement, or as operands to and, or, and not. To
determine whether a value x is truthy or falsy, Python applies bool(x), which always
returns True or False.
By default, instances of user-defined classes are considered truthy, unless either
`__bool__` or `__len__` is implemented. Basically, bool(x) calls x.`__bool__`() and uses
the result. If `__bool__` is not implemented, Python tries to invoke x.`__len__`(), and if
that returns zero, bool returns False. Otherwise bool returns True.
Our implementation of `__bool__` is conceptually simple: it returns False if the mag‐
nitude of the vector is zero, True otherwise. We convert the magnitude to a Boolean
using bool(abs(self)) because `__bool__` is expected to return a boolean.

## Chapter Summary
By implementing special methods, your objects can behave like the built-in types, en‐
abling the expressive coding style the community considers Pythonic.

A basic requirement for a Python object is to provide usable string representations of
itself, one used for debugging and logging, another for presentation to end users. That
is why the special methods `__repr__` and `__str__` exist in the data model.

Thanks to operator overloading, Python offers a rich selection of numeric types, from
the built-ins to decimal.Decimal and fractions.Fraction, all supporting infix arith‐
metic operators. Implementing operators, including reversed operators and augmented
assignment